# Federated Learning in Numpy

In this notebook, we show how to train a model across a distributed dataset using Numpy





# Step 1: Create Workers

In [1]:
import numpy as np
import syft as sy
import copy

x = sy.array([1,2,3])

bob = sy.VirtualWorker(id='bob')
alice = sy.VirtualWorker(id='alice')

# Step 2: Training Logic

In [2]:
def train(model, data, iters=100):
    
    syn0, syn1 = model
    X, y = data

    for j in range(iters):

        l1 = 1/(1+np.exp(-(X.dot(syn0))))
        l2 = 1/(1+np.exp(-(l1.dot(syn1))))

        l2_delta = (y - l2)*(l2*(1-l2))
        l1_delta = l2_delta.dot(syn1.transpose()) * (l1 * (1-l1))

        syn1 += l1.transpose().dot(l2_delta)
        syn0 += X.transpose().dot(l1_delta)
        
    return (syn0, syn1), ((y - l2)**2).sum()

def send(list_of_things, owner):
    pointers = list()
    for thing in list_of_things:
        pointers.append(thing.send(owner))
    return pointers

def get(list_of_pointers):
    things = list()
    for pointer in list_of_pointers:
        things.append(pointer.get())
    return things

# Step 2: Setup Example

This creates two datasets and sends them to Bob and Alice. This cell also initializes the model weights

In [4]:
X = sy.array([ [0,0,1],[0,1,1],[1,0,1],[1,1,1] ])
y = sy.array([[0,1,1,0]]).T

syn0 = sy.array(2*np.random.random((3,4)) - 1)
syn1 = sy.array(2*np.random.random((4,1)) - 1)

model = [syn0, syn1]

bobs_data = [copy.deepcopy(X), copy.deepcopy(y)]
alices_data = [copy.deepcopy(X), copy.deepcopy(y)]

bobs_data = send(bobs_data, bob)
alices_data = send(alices_data, alice)

# Step 3: Train

In [10]:
model = send(model, bob)

for iter in range(10):

    model, loss = train(model, bobs_data)

    bobs_loss = loss.get()

    model = send(get(model), alice)

    model, loss = train(model, alices_data)

    alices_loss = loss.get()

    model = send(get(model), bob)

    mean_loss = (bobs_loss + alices_loss)/2
    print(mean_loss)

0.8659159062491271
0.1740398697239179
0.028881457013389733
0.013600409537694418
0.008606615091287063
0.0062077510417679
0.004818547437833207
0.003919622602001533
0.0032935061242867175
0.002833928464672367
